# Gradient Boosting

Nós vamos utilizar o dataset Bank Marketing disponibilizado no [site da UCI](http://archive.ics.uci.edu/ml/datasets/Bank+Marketing). Utilizaremos uma versão adaptada para os objetivos da aula e disponível na pasta `data`.

> The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the investment product would be or not subscribed.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier, plot_metric, plot_tree, create_tree_digraph
from auto_lgbm import find_n_estimators, grid_search

from sklearn.metrics import log_loss
from evaluation import predictions_hist, confusion_matrix_report, grid_search_report

import shap
import matplotlib.pyplot as plt
import math

In [ ]:
sns.set_context("notebook", font_scale=1.5)
shap.initjs()

In [ ]:
df = pd.read_csv('../data/bank_marketing.csv')

Segue uma descrição sucinta de cada uma das colunas do dataset:

- `duration_seconds`: last contact duration, in seconds (numeric).

- `duration_minutes`: last contact duration, in minutes (numeric).

- `duration_hours`: last contact duration, in hours (numeric).

- `emp.var.rate`: employment variation rate - quarterly indicator (numeric)

- `nr.employed`: number of employees - quarterly indicator (numeric)

- `euribor3m`: euribor 3 month rate - daily indicator (numeric)

- `month`: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

- `contact`: contact communication type (1 for cellular, 2 for telephone) 

- `loan`: has personal loan? (0 for no, 1 for yes)

- `subscribed` - has the client subscribed a term deposit? (True, False)

## Preparando os dados

In [ ]:
X = df.drop(columns=['month', 'subscribed'])
y = df['subscribed']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=0)

## Aplicando o Gradient Boosting (implementação da LightGBM)

Utilize a classe `LGBMClassifier` para criar um modelo, treiná-lo e calcular as predições no dataset de teste.

Salve o modelo na variável `lgbm` e as predições em uma variável chamada `y_pred_proba`.

In [ ]:
lgbm = <code>

In [ ]:
<code>

In [ ]:
y_pred_proba[:5]

### Avaliando o modelo

Visualize as predições com a função `predictions_hist`, criada na aula de Regressão Logística, e defina um ponto de corte para a matriz de confusão.

As suas métricas de precisão e recall estão melhores do que as obtidas com o modelo de Regressão Logística?

In [ ]:
_ = predictions_hist(y_pred_proba, y_test, density=True)

In [ ]:
confusion_matrix_report(y_test, y_pred_proba, thres=<code>)

Utilize a função `log_loss` para calcular o log loss:

In [ ]:
<code>

### Visualização de árvores da LightGBM

In [ ]:
lgbm

Nós podemos ver acima que o nosso modelo foi treinado com `n_estimators=100` e `num_leaves=31`. Ou seja, o modelo compreende de 100 árvores de decisão, cada uma com até 31 folhas. Vamos plotar as 2 primeiras:

In [ ]:
# create_tree_digraph(lgbm, tree_index=0)
_ = plot_tree(lgbm, tree_index=0, figsize=(150, 50))

Plote a segunda árvore de decisão:

In [ ]:
<code>

## Modelando com features categóricas e early stopping

Utilize o atributo `dtypes` da classe `DataFrame` para identificar o tipo da coluna `month`:

In [ ]:
<code>

Para que a LightGBM trate a coluna `month` como uma feature categórica, é necessário que modiquemos seu tipo no Pandas:

In [ ]:
df['month'] = df['month'].astype('category')

Vamos então adicionar a coluna `month` e refazer o split entre treino e teste:

In [ ]:
X_cat = df.drop(columns=['subscribed'])

X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y, test_size=0.2, 
                                                                    random_state=0)

Para o early stopping, vamos quebrar o dataset de treino, que representa 80% dos dados, entre `dev` e `val`, de forma de `dev` fique com 60% dos dados e `val` com 20%. Ou seja, utilize a função `train_test_split` para criar `X_dev`, `X_val`, `y_dev` e `y_val`:

In [ ]:
validation_size = <code>

In [ ]:
<code>

In [ ]:
lgbm_es = LGBMClassifier(n_estimators=3000,
                         class_weight='balanced', random_state=0)

Agora vamos invocar o método `fit` da nossa LightGBM, utilizando `X_dev` e `y_dev` para treino, além de setar `eval_set` e `early_stopping_rounds`, de forma que a LightGBM utilize o dataset de validação para parar a adição de árvores depois de 50 iterações sem melhoria na métrica:

In [ ]:
<code>

In [ ]:
y_pred_proba_es = lgbm_es.predict_proba(X_test_cat)[:, 1]

In [ ]:
log_loss(y_test_cat, y_pred_proba_es)

## Tunando o parâmetro `max_depth` também

Defina a variável `max_depths` de forma que seja uma lista de `10` a `22`:

In [ ]:
max_n_estimators = 3000
early_stopping_rounds = 50
max_depths = <code>

In [ ]:
lgbm_md = LGBMClassifier(n_estimators=max_n_estimators, 
                         class_weight='balanced', random_state=0)

Preencha o código abaixo, de forma que a cada iteração do loop, seja treinada uma LightGBM com early stopping:

In [ ]:
results = pd.DataFrame(columns=['max_depth', 'best_n_estimators', 'best_log_loss'])

for max_depth in tqdm(max_depths):
    <code>
    
    results = results.append({'max_depth': max_depth, 
                              'best_n_estimators': lgbm_md.best_iteration_,
                              'best_log_loss': lgbm_md.best_score_['valid_0']['binary_logloss']},
                             ignore_index=True)

Na célula acima, é possível ver que a LightGBM salva o melhor valor encontrada para `n_estimators` no atributo `best_iteration_` e o seu desempenho em um dicionário que pode ser acessado pelo atributo `best_score_`.

Vamos então exibir os resultados:

In [ ]:
results['max_depth'] = results['max_depth'].astype(int)
results['best_n_estimators'] = results['best_n_estimators'].astype(int)

results

### Grid search com os melhores conjuntos de parâmetros

Vamos utilizar os melhores resultados obtidos na seção anterior e conferir via validação cruzada qual, de fato, é o melhor conjunto de hyper-parâmetros.

Para isso, a próxima célula cria grids de parâmetros. Note que isso é muito mais eficiente do que se criar somente um grid com todos esses parâmetros, já que o número de conjuntos de parâmetros a serem treinados passaria a ser a combinação de todos os valores (8*8=`64` ao invés de somente `8`).

In [ ]:
param_grids = [
    {'max_depth': [11], 'n_estimators': [325]},
    {'max_depth': [12], 'n_estimators': [266]},
    {'max_depth': [14], 'n_estimators': [306]},
    {'max_depth': [15], 'n_estimators': [260]},
    {'max_depth': [16], 'n_estimators': [349]},
    {'max_depth': [17], 'n_estimators': [389]},
    {'max_depth': [19], 'n_estimators': [321]},
    {'max_depth': [20], 'n_estimators': [319]},
]

No algoritmo de cross-validation do scikit-learn, quanto maior a métrica, melhor o modelo. Para poder-se utilizar esse tipo de algoritmo com métricas que tentam minimizar um erro (e.g., log loss), a métrica é negada, ou seja, multiplicada por `-1`.

A [lista de métricas pré-definidas](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules) pode ser encontrada na documentação do scikit-learn.

In [ ]:
scoring = 'neg_log_loss'

Agora, instancie a classe `GridSearchCV` com 3 folds de validação cruzada e o parâmetro `verbose=2`, além de passar outros parâmetros necessários:

In [ ]:
grid_search_cv = <code>

In [ ]:
grid_search_cv.fit(X_train_cat, y_train_cat, verbose=False)

A função `grid_search_report` foi criada por nós para facilitar a visualização dos resultados:

In [ ]:
grid_search_report(grid_search_cv.grid_scores_, scoring, scoring_alias='log_loss')

In [ ]:
grid_search_cv.best_estimator_

In [ ]:
lgbm_md_best = grid_search_cv.best_estimator_

In [ ]:
y_pred_proba_md = lgbm_md_best.predict_proba(X_test_cat)[:, 1]
log_loss(y_test_cat, y_pred_proba_md)

In [ ]:
# create_tree_digraph(lgbm, tree_index=2)
_ = plot_tree(lgbm_md_best, tree_index=2, figsize=(150, 50))

## Incluindo tunagem do parâmetro `learning_rate`

In [ ]:
lgbm_lr = LGBMClassifier(class_weight='balanced', verbose=-1)

Vamos testar os seguintes valores de `learning_rate`:

In [ ]:
learning_rates = [0.01, 0.03, 0.1]

Perceba que os nomes das métricas diferem entre sklearn e LightGBM. Enquanto no sklearn utilizamos `neg_log_loss` para validação cruzada, na LightGBM utilizamos `binary_logloss` pro early stopping:

In [ ]:
n_estimators_result = find_n_estimators(lgbm_lr, X_train_cat, y_train_cat,
                                        eval_metric='binary_logloss',
                                        learning_rates=learning_rates,
                                        max_depths=range(12, 23),
                                        random_state=0)

In [ ]:
n_estimators_result.sort_values(by='best_score').head(10)

A partir dos resultados acima, selecionaremos apenas 3 conjuntos de parâmetros, cada um com `learning_rate` diferente, de forma que o grid search não demore muito. Fique à vontade para fazer um grid search mais completo em casa.

No mais, note que, quanto menor o `learning_rate`, maior é o `n_estimators`. Por consequência, maior também serão os tempos de treino e de predição.

In [ ]:
param_grids = [
    {'learning_rate': [0.01], 'max_depth': [16], 'n_estimators': [3855]}, 
    {'learning_rate': [0.03], 'max_depth': [20], 'n_estimators': [1070]}, 
    {'learning_rate': [0.1], 'max_depth': [20], 'n_estimators': [319]}
]

Criamos a função `grid_search` para facilitar o procedimento:

In [ ]:
grid_search_result, lgbm_lr_best = grid_search(lgbm_lr, X_train_cat, y_train_cat, 
                                               param_grids, scoring='neg_log_loss',
                                               scoring_alias='log_loss')

In [ ]:
grid_search_result

In [ ]:
lgbm_lr_best

In [ ]:
y_pred_proba_lr = lgbm_lr_best.predict_proba(X_test_cat)[:, 1]
log_loss(y_test_cat, y_pred_proba_lr)

Os resultados ficaram melhores?

Um modelo de gradient boosting com esses 3 hyper-parâmetros tunados em geral tem um bom trade-off entre o tempo necessário para se descobrir os melhores parâmetros e a qualidade das predições.

## Interpretando o modelo

Para interpretar o modelo, utilizaremos o [SHAP](https://github.com/slundberg/shap).

Para não despendermos muito tempo calculando os SHAP values, utilizaremos uma amostra aleatória de 100 exemplos:

In [ ]:
X_train_sample = X_train_cat.sample(1_000, random_state=0)

O `TreeExplainer`, ao contrário do `KernelExplainer` que é genérico, é otimizado para modelos baseados em árvores e tem suporte à LightGBM.

In [ ]:
explainer = shap.TreeExplainer(lgbm_lr_best, data=X_train_sample)
shap_values = explainer.shap_values(X_train_sample)

Os SHAP values são nada mais do que a contribuição de cada variável para cada predição. A sua unidade é sempre a mesma unidade do target.

Com os SHAP values calculados, vamos plotar os feature importances:

In [ ]:
shap.summary_plot(shap_values, X_train_sample, plot_type='bar')

Modifique o parâmetro `plot_type='bar'` para `plot_type='dot'`. Veja que agora é possível fazer uma análise mais aprofundada com o summary plot:

In [ ]:
<code>

### Vamos dar um deep dive em algumas features

Pense no `dependence_plot` como um zoom que nós podemos dar para entender o que o modelo aprendeu com relação a uma feature específica:

In [ ]:
def dependence_plot(feature, show=True):
    shap.dependence_plot(feature, shap_values, X_train_sample, 
                         interaction_index=feature, show=show)

In [ ]:
# show=False e o xlim na linha de baixo é para não exibirmos outliers
dependence_plot('duration_seconds', show=False)
_ = plt.xlim(0, 1600)

In [ ]:
dependence_plot('emp.var.rate')

O nome `dependence plot` é dado ao fato de que o SHAP automaticamente identifica qual é a variável que mais interage com a variável em questão e exibe o seu comportamento dependendo dessa outra variável.

O parâmetro `interaction_index` nos permite selecionar qualquer outra variável para essa comparação. O SHAP é uma ferramenta poderosa, mas ainda é recente e não tem suporte à features categóricas, assim como tem a LightGBM. Dessa forma, setamos o `interaction_index` para a própria feature, de forma a não dar erro.

In [ ]:
feature = 'euribor3m'
shap.dependence_plot(feature, shap_values, X_train_sample, 
                     interaction_index=feature)

In [ ]:
feature = 'nr.employed'
shap.dependence_plot(feature, shap_values, X_train_sample, 
                     interaction_index=feature)

### Extra: vamos analisar algumas predições

Lembram da função logística (a.k.a., sigmoid)?

In [ ]:
def logistic(x):
    return math.exp(x) / (math.exp(x) + 1)

In [ ]:
explainer.expected_value, logistic(explainer.expected_value)

A célula de cima nos diz que o valor esperado (ou seja a média) das predições no dataset de teste é ~0.65%.

O `force_plot` é uma analogia à física. As variáveis em vermelho "empurram" o valor da predição para ser mais alto, quanto as variáveis em azul "empurram" o valor da predição para baixo.

In [ ]:
def force_plot(id_):
    return shap.force_plot(explainer.expected_value, shap_values[id_, :], 
                           X_train_sample.iloc[id_, :])

Uma ótima avaliação de um classificador pode ser analisar os force plots dos falsos positivos scores mais altos e os falsos negativos com scores mais baixos. Essa estratégia tem o potencial de 1) ajudar a identificar features relevantes que não estão no modelo; e/ou 2) fazer com que stakeholders ganhem confiança no modelo caso os erros façam sentido.

Vamos aplicar o `force_plot` à primeira observação da amostra aleatória:

In [ ]:
force_plot(id_=0)

Para facilitar a interpretação do valor predito, basta aplicar a função logística:

In [ ]:
logistic(-8.84)

In [ ]:
force_plot(id_=1)

In [ ]:
logistic(1.6)